In [3]:
!pip install gradio
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.4 MB/

In [4]:
import gradio as gr
import re

In [5]:
from simpletransformers.language_generation import LanguageGenerationModel,LanguageGenerationArgs
Language_gen_args = LanguageGenerationArgs()
Language_gen_args.max_length=40
Language_gen_args.use_cuda=False

In [7]:
model=LanguageGenerationModel("gpt2",'/content/drive/MyDrive/CommentaryGeneration/Trial1/output',args=Language_gen_args,use_cuda=False)

In [8]:
output=model.generate("SIX runs")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [9]:
output[0]

"SIX runs, it's too predictable. the bowler Kumar finishes with a six. On a length and outside off, bang into the stands. KKR back in this. Cracking finish from Umesh."

In [10]:
result=output[0]
commentary=re.split(r"\.\s*",result)
for i in range (1,len(commentary)):
  if(re.match(r'[1-6]|FOUR|SIX|OUT|no\s\run',commentary[i])!="NONE"):
    index=i

result=commentary[0]
for i in range (1,index):
  result=result+". "+commentary[i]

result

"SIX runs, it's too predictable. the bowler Kumar finishes with a six. On a length and outside off, bang into the stands. KKR back in this. Cracking finish from Umesh"

In [11]:
def generatecommentary(bowler,batsman,runs_scored,extra):
  index = 1;
  prompt = str(bowler)+" to "+str(batsman)
  event = generateevent(runs_scored,extra)
  output=model.generate(event)

  result=output[0]
  commentary=re.split(r"\.\s*",result)
  for i in range (1,len(commentary)):
    if(re.match(r'[1-6]|FOUR|SIX|OUT|no|one\s\run',commentary[i])!="NONE"):
      index=i

  result=commentary[0]
  for i in range (1,index-1):
    result=result+". "+commentary[i]

  res=prompt+" "+result
  return res

In [12]:
def generateevent(runs_scored,extra):
  if(runs_scored==0):
    event="no run"

  if((extra=="no extra" or extra=="no ball") and runs_scored!=0 ):
    if(extra=="no ball"):
      event="(no ball) "
    else:
      event = ""
    if(runs_scored=="OUT"):
      event += "OUT"
    elif(runs_scored=="1"):
      event +="one run"
    elif(runs_scored=="2" or runs_scored=="3" or runs_scored=="5"):
      event+=runs_scored+" runs"
    elif(runs_scored=="4"):
      event+="FOUR runs"
    elif(runs_scored=="6"):
      event+="SIX runs"

  else:
    if(runs_scored!="0" or runs_scored!="6"):
      event=runs_scored+" "+extra
  return event

In [14]:
input = [
    gr.Textbox(label="Bowler"),
    gr.Textbox(label="Batsman"),
    gr.Dropdown(["OUT","0", "1","2","3","4","5","6"], label="Runs scored (including extras)"),
    gr.Dropdown(["no extra","wide","no ball","leg bye"], label="Extra_type")
]

In [16]:
commentarygenerator = gr.Interface(
    fn=generatecommentary,
    inputs=input,
    outputs=gr.Textbox(label="Generated Commentary")
)

In [18]:
commentarygenerator.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8c2abad5c8f2823678.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8c2abad5c8f2823678.gradio.live
